# Лабораторная работа №1

## Задание:

In [1]:
from IPython.display import display, Markdown

with open('lab01.md', 'r') as fh:
    content = fh.read()

display(Markdown(content))

### К лекции №2

**Упражнение №5.0.1**  

Освойтесь с NumPy с помощью этого туториала:  

https://jakevdp.github.io/PythonDataScienceHandbook/02.00-introduction-to-numpy.html
  
Перед следующим упражнением, убедитесь что вы понимаете разницу между nparray и стандартным list!  
  
**Упражнение №5.0.2**  
Изучите Pandas - дефакто стандарт для работы с табличными (не bigdata) данными.  

https://jakevdp.github.io/PythonDataScienceHandbook/03.01-introducing-pandas-objects.html

https://jakevdp.github.io/PythonDataScienceHandbook/03.02-data-indexing-and-selection.html

https://jakevdp.github.io/PythonDataScienceHandbook/03.01-introducing-pandas-objects.html 
 
**Упражнение №5.1**  

Сделайте свой датафрейм на основе 2D-nparray  
Он должен поддерживать:  
* построчный индекс  
* хранить названия и порядок колонок  
* функции индексирования:  
 * iloc[i,j] - получить элемент по i-й строке j-й колонке. (i/j также могут быть slice-объектами)  
 * loc[k,z] - получить элемент по k-му значению построчного индекса z-й колонке (k может быть iterable/slice, z - iterable)  
 * [] - переопределить getitem, сделать его алиасом вашей loc-функции  
Также нужно сделать статический метод, читающий CSV:   
`your_package.read_csv(file_path, **csv_opts)`   
и возвращающий инстанс вашего датафрейма с прочитанными данными.    
Метод должен принимать параметры, настраивающие поведение чтения CSV-файла.  

**Упражнение №6.1**

Почитайте про случайный лес в [мануалах scikit](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier)

Научитесь обучать лес и запускать его.  
Для хранения и передачи данных для sklearn вы должны использовать собственный датафрейм!  

**Упражнение №6.2**

Научитесь визуализировать деревья. Почитайте:

  * [sklearn.tree.export_graphviz](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html#sklearn.tree.export_graphviz)
  * [How to Visualize a Decision Tree from a Random Forest in Python using Scikit-Learn](https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c)

**Задача №7**

Посмотрите на данные из выборки ещё раз. Обучите RF на этих данных.
Сравните RF c алгоритмами **p1_fraud**, **p2_fraud** и **p3_fraud**.

**Задача №8**

Подумайте, как с помощью RF оценивать информативность признаков?
Найдите информативные признаки. 

Сравните результаты с результатами **упражнения №1**.


## Упражнение №5.0.1

## Упражнение №5.0.2

## Упражнение №5.1

In [63]:
import csv
import numpy as np

from typing import Iterable, Sequence, Optional, Union


class IdxView:
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __getitem__(self, key):
        pass
    
    def __setitem__(self, key, value):
        pass


class Dataframe:
    def __init__(self,
                 data: np.ndarray,
                 columns: Union[np.ndarray, Sequence, None] = None,
                 indices: Union[np.ndarray, Sequence, None] = None):
        """
        Two-dimensional, size-immutable table.
        
        Contains labeled axis. Provides O(1) rows & columns access via index or key.
        
        Parameters
            :data: ndarray
                Contains rows
            :columns: ndarray, sequence* or None
                Contains column labels. :columns: len is required to equal data.shape[1]
                If :columns: is None labels are filled via np.arrange on data.shape[1]
            :indices: ndarray, sequence* or None
                Contains row labels. :indices: len is required to equal data.shape[0]
                If :indices: is None labels are filled via np.arrange on data.shape[0]
            
            *sequence (vs iterable) allows __len__, making shape checks easier.
             Also removes intermidiate array creations for preserving one-time iterables entirely.
            
        Raises
            :ValueError:
                if columns/indices shape requirements are not satisfied or recieved non-2D array as data.
        """
        if len(data.shape) != 2:
            raise ValueError(f"2D array required. "
                             f"Recieved array of {len(data.shape)} dimensions")
        # Transposed due to np.array type-casting requirements
        self.data = self.data.T
        x, y = self.data.shape
        
        def validate_shape(seq, req_len, label: str):
            if seq is None:
                return np.arange(req_len)
            if len(seq) != req_len:
                raise ValueError(f"Actual {label}s and recieved {label}s amount vary: "
                                 f"{req_len} vs {len(seq)}")
            return np.array(seq)
        
        columns = validate_shape(columns, x, 'column')
        indices = validate_shape(indices, y, 'row')
        
        # Construct two-way mapping for loc & iloc internal usage
        self.col_to_loc = {col: loc for col, loc in enumerate(columns)}
        self.loc_to_col = np.array(columns)
        self.idx_to_loc = {idx: loc for idx, loc in enumerate(indices)}
        self.loc_to_idx = np.array(indices)
    
    @property
    def loc(self):
        pass
    
    @property
    def iloc(self):
        pass
    
    def to_csv(self, file_path, **kwargs):
        with open(file_path, 'w', newline='') as file:
            csvfile = csv.writer(file, **kwargs)
            csvfile.writerow(self.columns)
            csvfile.writerows(self.data.T)
    
    @staticmethod
    def read_csv(file_path, **kwargs):
        with open(file_path, 'r', newline='') as file:
            csvfile = csv.reader(file, **kwargs)
            try: columns = next(csvfile)
            except StopIteration: raise RuntimeError("File is empty")
            return Dataframe(np.ndarray(csvfile), columns)
    
    __getitem__ = loc